## CHAPTER 18
---
# NAIVE BAYES

---
Bayes' theorem is the premier method for understanding the probability of some event, $P(A|B)$, given some new information, $P(B|A)$, and a prior belief in the probability of the event, P(A):$$
P(A | B) = \frac{P(B|A)P(A)}{P(B)}
$$

The Bayesian method's popularity has skyrocked in the last decade, more and more rivaling the traditional frequentist applications in academia, government, and business. In machine learning, one applicaiton of Bayes' theorem to classifican comes in the form of the naive Bayes classifier. Naive Bayes classifiers combine a number of desirable qualities in practical machine learning into a single classifier:

1. An intuitive approach
2. The ability to work with small data
3. Low computation costs for training and prediction
4. Often solid results in a variety of settigns.

Specifically, a naive Bayes classifier is based on:$$
P(y | x_1, ..., x_j) = \frac{P(x_1, ..., x_j | y)P(y)}{P(x_1,...,x_j)}
$$
where,

- $P(y | x_1, ..., x_j)$ is called the *posterior* and is the probability that an observation is class y given observation's values for the j features, $x_1, ..., x_j$
- $P(x_1, ..., x_j)$ is called likelihood and is the *likelihood* of an observation's values for features, $x_1, ..., x_j$, given their class y.
- $P(y)$ is called the prior and is our belief for the probability of class y before looking at the data
- $P(x_1, ..., x_j$) is called the *marginal probability*.

## 18.1 Training a Classifier for Continuous Features

- You have only continuous features and you want to train a naive Bayes classifier.
- Use a **Gaussian Naive Bayes classifier** in scikit-learn

In [1]:
# Load libraries
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB

# Load data
iris = datasets.load_iris()
features = iris.data
target = iris.target

# Create Gaussian Naive Bayes object
classifer = GaussianNB()

# Train model
model = classifer.fit(features, target)

# Create new observation
new_observation = [[ 4,  4,  4,  0.4]]

# Predict class
model.predict(new_observation)

array([1])

#### Discussion:
The most common type of naive Bayes classifier is the Gaussian naive Bayes, in which we assume that the likelihood of the feature values, x, given an observation is of class y, follows a normal distribution:
$$
p(x_j | y) = \frac{1}{\sqrt{2\pi \sigma_y^2}} e^{-\frac{(x_j - \mu_y)^2}{2\sigma_y^2}}
$$
where $\sigma_y^2$ and $\mu_y$ are the variance and mean values of feature x_j for class y. Because of the assumption of the normal distribution, Gaussian naive Bayes is *best used in cases when all our features are continuous*.

One of the interesting aspects of naive Bayes classifiers is that they allow us to assign a prior belief over the respect target classes. We can do this using GaussianNB's priors parameter, which takes in a list of the probabilities assigned to each class of the target vector:

In [2]:
# Create Gaussian Naive Bayes object with prior probabilities of each class
clf = GaussianNB(priors=[0.25, 0.25, 0.5])

# Train model
model = classifer.fit(features, target)

## 18.2 Training a Classifier for Discrete and Count Features

- Given discrete or count data, you need to train a naive Bayes classifier.
- Use a **Multinomial Naive Bayes classifier**

In [3]:
# Load libraries
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

# Create text
text_data = np.array(['I love Brazil. Brazil!',
                      'Brazil is best',
                      'Germany beats both'])

# Create bag of words
count = CountVectorizer()
bag_of_words = count.fit_transform(text_data)

# Create feature matrix
features = bag_of_words.toarray()

# Create target vector
target = np.array([0,0,1])

# Create multinomial naive Bayes object with prior probabilities of each class
classifer = MultinomialNB(class_prior=[0.25, 0.5])

# Train model
model = classifer.fit(features, target)

# Create new observation
new_observation = [[0, 0, 0, 1, 0, 1, 0]]

# Predict new observation's class
model.predict(new_observation)

array([0])

#### Discussion:
Multinomial naive Bayes works similarly to Gaussian naive Bayes, but the features are assumed to be multinomial distributed. In practice this means that this classifier is *commonly used when we have discrete data*. One of the most common uses is text classification using bag of words or tf-idf approaches.

MultinomialNB contains an additive smoothing hyperparameter, alpha, that should be tuned. The default value is 1.0, with 0.0 meaning no smoothing takes place.

## 18.3 Training a Naive Bayes Classifier for Binary Features

- You have binary feature data and need to train a naive Bayes classifier.
- Use a **Bernoulli Naive Bayes classifier**

In [4]:
# Load libraries
import numpy as np
from sklearn.naive_bayes import BernoulliNB

# Create three binary features
features = np.random.randint(2, size=(100, 3))

# Create a binary target vector
target = np.random.randint(2, size=(100, 1)).ravel()

# Create Bernoulli Naive Bayes object with prior probabilities of each class
classifer = BernoulliNB(class_prior=[0.25, 0.5])

# Train model
model = classifer.fit(features, target)

The Bernoulli naive Bayes classifier assumes that all our features are binary such that they take only two values (e.g. a nominal categorical feature that has been one-hot encoded). Like its multinomial cousin, Bernoulli naive Bayes is *often used in text classification*, when our feature matrix is simply the presence or absence of a word in a document

## 18.4 Calibrating Predicted Probabilities

- You want to calibrate the predicted probabilities from naive Bayes classifiers so they are interpretable.
- Use **CalibratedClassifierCV**

In [5]:
# Load libraries
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV

# Load data
iris = datasets.load_iris()
features = iris.data
target = iris.target

# Create Gaussian Naive Bayes object
classifer = GaussianNB()

# Create calibrated cross-validation with sigmoid calibration
classifer_sigmoid = CalibratedClassifierCV(classifer, cv=2, method='sigmoid')

# Calibrate probabilities
classifer_sigmoid.fit(features, target)

# Create new observation
new_observation = [[ 2.6,  2.6,  2.6,  0.4]]

# View calibrated probabilities
classifer_sigmoid.predict_proba(new_observation)

array([[0.31859969, 0.63663466, 0.04476565]])

#### Discussion:
- In Naive Bayes, while the ranking of predicted probabilities for the different target classes is valid, the raw predicted probabilities tend to take on extreme values close to 0 and 1.
- To obtain meaningful predicted probabilities we need conduct calibration using the CalibratedClassifierCV class, which creates well-calibrated predicted probabilities using k-fold cross-validation. 
- In CalibratedClassifierCV the training sets are used to train the model and the test set is used to calibrate the predicted probabilities. The returned predicted probabilities are the average of the k-folds.

In [6]:
# Extreme probabilities from a Gaussian naive Bayes 
classifer.fit(features, target).predict_proba(new_observation)

array([[2.31548432e-04, 9.99768128e-01, 3.23532277e-07]])

In [7]:
# Calibrated probabilities from sigmoid calibration
classifer_sigmoid.predict_proba(new_observation)

array([[0.31859969, 0.63663466, 0.04476565]])